<a href="https://www.kaggle.com/code/vtrackstar/ml-nba-offensive-playstyle-through-11-22-24?scriptVersionId=209409050" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Ask
* Goals
   * Cluster NBA players into distinct offensive archetypes based on their playing styles and position data.
   * Analyze player trends by identifying specific archetypes (e.g., scoring guards, playmaking guards, etc.) and how common they are within the NBA player pool.
* Data Needed
   * Player statistics from the current NBA season, including data points like points, assists, field goal attempts, and the percentage of time spent at different positions (which can be extracted from Basketball-Reference or similar datasets).
   * Player positions for each game or season to accurately assess the percentage of time each player spends at various positions (e.g., PG, SG, SF, PF, C).

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


pd.set_option('display.max_columns', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Shooting.csv
/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - PerGame.csv
/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Play by Play.csv
/kaggle/input/updated/2024 NBA Scoring (11_22) - PerGame (1).csv


## Prepare

In [2]:
perg = pd.read_csv('/kaggle/input/updated/2024 NBA Scoring (11_22) - PerGame (1).csv')
shooting = pd.read_csv('/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Play by Play.csv')
playbplay = pd.read_csv('/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Shooting.csv')

In [3]:
perg

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
0,1,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,9.9,12.0,6.4,0.6,1.4,3.2,3.0,32.4,NaN
1,2,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,8.6,11.2,2.8,1.2,1.9,2.1,1.9,31.3,NaN
2,3,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,9.4,14.0,11.5,1.5,0.9,4.0,1.8,30.0,NaN
3,4,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,7.8,8.2,6.1,1.4,0.5,2.9,2.6,29.1,NaN
4,5,Paolo Banchero,22,ORL,PF,5,5,36.4,9.6,19.4,0.495,2.2,6.4,0.344,7.4,13.0,0.569,0.552,7.6,11.8,0.644,2.4,6.4,8.8,5.6,0.6,0.8,2.2,2.6,29.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,Jamal Cain,25,NOP,SF,12,0,15.3,1.6,3.7,0.432,0.8,2.0,0.375,0.8,1.7,0.500,0.534,0.8,1.2,0.643,0.9,2.0,2.9,0.8,0.4,0.4,0.6,1.5,4.7,NaN
296,297,Drew Eubanks,27,UTA,C,12,0,14.9,1.9,3.0,0.639,0.1,0.3,0.333,1.8,2.8,0.667,0.653,0.8,1.3,0.600,1.4,2.8,4.3,1.2,0.3,0.7,0.8,1.3,4.7,NaN
297,298,Taylor Hendricks,21,UTA,PF,3,3,25.0,1.3,6.0,0.222,1.0,4.0,0.250,0.3,2.0,0.167,0.306,1.0,1.3,0.750,1.7,3.3,5.0,0.7,1.7,1.3,0.7,2.7,4.7,NaN
298,299,Reggie Jackson,34,PHI,PG,7,0,13.6,1.7,4.7,0.364,1.3,3.1,0.409,0.4,1.6,0.273,0.500,0.0,0.0,NaN,0.0,1.0,1.0,2.0,0.6,0.1,0.7,1.0,4.7,NaN


In [4]:
shooting.head()

,Rk,Player,Age,Team,Pos,G,GS,MP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPassTO,LostBallTO,ShootFoulCmtd,OffFoulCmtd,ShootFoulDrawn,OffFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,Awards
0,1,Cade Cunningham,23,DET,PG,17,17,618,92.0,8.0,0.0,0.0,0.0,-1.7,-1.2,49,22,18,3,37,2,365,9,24,NaN
1,2,De'Aaron Fox,27,SAC,PG,16,16,604,91.0,9.0,0.0,0.0,0.0,2.9,2.5,21,29,13,4,55,10,228,8,17,NaN
2,3,James Harden,35,LAC,PG,17,17,593,60.0,40.0,0.0,0.0,0.0,-0.3,-6.1,37,27,13,9,46,3,344,6,18,NaN
3,4,Keegan Murray,24,SAC,PF,16,16,590,0.0,0.0,16.0,83.0,0.0,3.7,5.5,9,6,26,1,11,4,70,2,5,NaN
4,5,Devin Booker,28,PHO,SG,16,16,588,1.0,37.0,56.0,7.0,0.0,-4.6,-8.4,11,13,11,4,39,6,263,5,18,NaN


In [5]:
playbplay.head()

,Rk,Player,Age,Team,Pos,G,GS,MP,FG%,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,%3PA_Heaves,3P%_Heaves,Num_Heaves_Att,Num_Heaves_Made,Awards
0,1,Cade Cunningham,23,DET,PG,17,17,618,0.438,12.8,0.688,0.212,0.291,0.129,0.056,0.312,0.474,0.528,0.455,0.455,0.421,0.358,0.243,0.605,0.021,3,0.075,0.500,1,0,NaN
1,2,De'Aaron Fox,27,SAC,PG,16,16,604,0.506,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015,5,0.150,0.267,1,0,NaN
2,3,James Harden,35,LAC,PG,17,17,593,0.365,16.8,0.502,0.151,0.196,0.144,0.011,0.498,0.419,0.561,0.321,0.385,0.667,0.311,0.175,0.524,0.007,2,0.037,0.400,0,0,NaN
3,4,Keegan Murray,24,SAC,PF,16,16,590,0.426,16.6,0.497,0.137,0.191,0.109,0.060,0.503,0.549,0.920,0.400,0.400,0.455,0.304,0.640,1.000,0.082,14,0.337,0.387,0,0,NaN
4,5,Devin Booker,28,PHO,SG,16,16,588,0.438,16.6,0.580,0.101,0.177,0.160,0.142,0.420,0.509,0.690,0.471,0.457,0.488,0.339,0.435,0.707,0.010,2,0.116,0.286,0,0,NaN


In [6]:
print(perg.columns)
print(shooting.columns)
print(playbplay.columns)

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Awards'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PG%', 'SG%',
       'SF%', 'PF%', 'C%', 'OnCourt', 'On-Off', 'BadPassTO', 'LostBallTO',
       'ShootFoulCmtd', 'OffFoulCmtd', 'ShootFoulDrawn', 'OffFoulDrawn',
       'PtsGenbyAsts', 'And1s', 'FGA_Blkd', 'Awards'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       'AvgFGDist', '%FGA_2P', '%FGA_0-3', '%FGA_3-10', '%FGA_10-16',
       '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3', 'FG%_3-10', 'FG%_10-16',
       'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_Dunks',
       'Num_Dunks', '%3PA_Heaves', '3P%_Heaves', 'Num_Heaves_Att',
       'Num_Heaves_Made', 'Awards'],
      dtype='object')


In [7]:
nullperg = perg.isnull().sum()
nullsht = shooting.isnull().sum()
nullpbp = playbplay.isnull().sum()
print(nullperg)
print(nullsht)
print(nullpbp)

Rk          0
Player      0
Age         0
Team        0
Pos         0
G           0
GS          0
MP          0
FG          0
FGA         0
FG%         0
3P          0
3PA         0
3P%        17
2P          0
2PA         0
2P%         0
eFG%        0
FT          0
FTA         0
FT%         3
ORB         0
DRB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PF          0
PTS         0
Awards    300
dtype: int64
Rk                  0
Player              0
Age                 0
Team                0
Pos                 0
G                   0
GS                  0
MP                  0
PG%                 1
SG%                 1
SF%                 1
PF%                 1
C%                  1
OnCourt             0
On-Off              0
BadPassTO           0
LostBallTO          0
ShootFoulCmtd       0
OffFoulCmtd         0
ShootFoulDrawn      0
OffFoulDrawn        0
PtsGenbyAsts        0
And1s               0
FGA_Blkd            0
Awards            400
dty

In [8]:
pergclean = perg.fillna(0)
shtclean = shooting.fillna(0)
pbpclean = playbplay.fillna(0)

In [9]:
nullperg1 = pergclean.isnull().sum()
nullsht1 = shtclean.isnull().sum()
nullpbp1 = pbpclean.isnull().sum()
print(nullperg1.head())
print(nullsht1.head())
print(nullpbp1.head())

Rk        0
Player    0
Age       0
Team      0
Pos       0
dtype: int64
Rk        0
Player    0
Age       0
Team      0
Pos       0
dtype: int64
Rk        0
Player    0
Age       0
Team      0
Pos       0
dtype: int64


In [10]:
pergame = pergclean[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'AST', 'TOV', 'PTS']]
shot = shtclean[['Player', 'PG%', 'SG%',
       'SF%', 'PF%', 'C%', 'BadPassTO', 'LostBallTO',
       'OffFoulCmtd', 'ShootFoulDrawn',
       'PtsGenbyAsts', 'And1s', 'FGA_Blkd']]
playbyplay = pbpclean[['Player', 'AvgFGDist', '%FGA_2P', '%FGA_0-3', '%FGA_3-10', '%FGA_10-16',
       '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3', 'FG%_3-10', 'FG%_10-16',
       'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_Dunks',
       'Num_Dunks']]
merged1 = pd.merge(pergame, shot, on='Player', how='left').merge(playbyplay, on='Player', how='left')

In [11]:
merged2 = merged1[merged1['G'] >= 8]
merged = merged2.copy()
merged

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks
0,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,6.4,3.2,32.4,0.0,0.0,0.0,63.0,37.0,15.0,16.0,11.0,84.0,254.0,27.0,19.0,6.5,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.379,0.000,0.190,59.0
1,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,2.8,2.1,31.3,0.0,0.0,0.0,6.0,94.0,14.0,10.0,2.0,60.0,95.0,15.0,14.0,8.2,0.888,0.378,0.326,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0
2,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,11.5,4.0,30.0,0.0,0.0,0.0,0.0,100.0,22.0,17.0,2.0,28.0,303.0,3.0,14.0,9.2,0.808,0.280,0.453,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0
3,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,6.1,2.9,29.1,0.0,2.0,48.0,51.0,0.0,23.0,15.0,6.0,45.0,256.0,8.0,14.0,17.9,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.333,0.492,0.050,15.0
5,LaMelo Ball,23,CHO,PG,15,15,33.1,9.9,23.1,0.428,4.5,12.5,0.356,5.4,10.5,0.513,0.525,4.7,5.3,0.886,1.0,6.7,4.2,28.9,73.0,26.0,1.0,0.0,0.0,35.0,11.0,8.0,33.0,253.0,7.0,15.0,17.6,0.457,0.173,0.182,0.081,0.020,0.543,0.513,0.617,0.492,0.429,0.143,0.356,0.235,0.567,0.009,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Dalen Terry,22,CHI,SG,17,0,13.8,1.6,3.2,0.491,0.5,1.1,0.444,1.1,2.2,0.514,0.564,1.2,1.6,0.778,0.8,1.1,1.0,4.9,2.0,21.0,72.0,5.0,0.0,9.0,3.0,0.0,12.0,48.0,1.0,4.0,10.0,0.673,0.527,0.109,0.000,0.036,0.327,0.514,0.552,0.333,0.000,0.500,0.444,0.579,1.000,0.055,3.0
293,Jared Butler,24,WAS,SG,10,0,8.2,1.7,3.4,0.500,0.1,0.5,0.200,1.6,2.9,0.552,0.515,1.3,1.9,0.684,0.2,2.4,0.3,4.8,70.0,31.0,0.0,0.0,0.0,2.0,0.0,1.0,8.0,52.0,4.0,8.0,8.6,0.853,0.382,0.382,0.059,0.029,0.147,0.552,0.615,0.462,1.000,0.000,0.200,0.188,1.000,0.000,0.0
295,Jamal Cain,25,NOP,SF,12,0,15.3,1.6,3.7,0.432,0.8,2.0,0.375,0.8,1.7,0.500,0.534,0.8,1.2,0.643,0.9,0.8,0.6,4.7,0.0,30.0,52.0,17.0,0.0,5.0,0.0,1.0,7.0,22.0,2.0,0.0,15.7,0.455,0.205,0.182,0.045,0.023,0.545,0.500,0.778,0.375,0.000,0.000,0.375,0.800,1.000,0.000,0.0
296,Drew Eubanks,27,UTA,C,12,0,14.9,1.9,3.0,0.639,0.1,0.3,0.333,1.8,2.8,0.667,0.653,0.8,1.3,0.600,1.4,1.2,0.8,4.7,0.0,0.0,0.0,10.0,90.0,3.0,4.0,1.0,7.0,35.0,0.0,1.0,7.1,0.917,0.333,0.472,0.111,0.000,0.083,0.667,0.750,0.588,0.750,0.000,0.333,0.818,0.000,0.194,6.0


In [12]:
duplicate_count = merged.duplicated().sum()

# Print the result
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [13]:
conditions = [
    (merged['C%'] > 50) | ((merged['PF%'] > 65) & (merged['SF%'] < 5)),
    (merged['PG%'] > 50) | ((merged['SG%'] > 1) & (merged['SF%'] < 6)),
]

choices = ['Bigs', 'Guards']
merged['Position Group'] = np.select(conditions, choices, default='Wings')

In [14]:
bigs1 = merged[merged['Position Group'] == 'Bigs']
guards1 = merged[merged['Position Group'] == 'Guards']
wings1 = merged[merged['Position Group'] == 'Wings']
bigs = bigs1.copy()
guards = guards1.copy()
wings = wings1.copy()

In [15]:
bigs['%FGA_0-10'] = bigs['%FGA_0-3'] + bigs['%FGA_3-10']
bigs.head()

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_0-10
1,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,2.8,2.1,31.3,0.0,0.0,0.0,6.0,94.0,14.0,10.0,2.0,60.0,95.0,15.0,14.0,8.2,0.888,0.378,0.326,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0,Bigs,0.704
2,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,11.5,4.0,30.0,0.0,0.0,0.0,0.0,100.0,22.0,17.0,2.0,28.0,303.0,3.0,14.0,9.2,0.808,0.280,0.453,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0,Bigs,0.733
10,Kevin Durant,36,PHO,PF,9,9,38.8,9.9,17.9,0.553,2.7,6.2,0.429,7.2,11.7,0.619,0.627,5.1,6.1,0.836,0.2,3.4,3.3,27.6,0.0,0.0,0.0,84.0,16.0,17.0,6.0,1.0,26.0,76.0,9.0,3.0,15.0,0.652,0.155,0.130,0.292,0.075,0.348,0.619,0.920,0.667,0.468,0.500,0.429,0.492,0.750,0.037,6.0,Bigs,0.285
11,Karl-Anthony Towns,29,NYK,C,14,14,33.0,9.6,17.5,0.551,2.6,5.3,0.500,7.0,12.2,0.573,0.627,4.9,5.7,0.850,2.8,2.9,1.9,26.8,0.0,0.0,0.0,0.0,100.0,8.0,7.0,8.0,43.0,93.0,14.0,18.0,11.2,0.698,0.343,0.278,0.061,0.016,0.302,0.573,0.631,0.559,0.467,0.000,0.500,0.541,0.973,0.073,17.0,Bigs,0.621
27,Jaren Jackson Jr.,25,MEM,C,14,14,28.9,8.2,15.6,0.528,1.9,5.2,0.356,6.4,10.4,0.614,0.587,4.6,5.8,0.802,1.3,1.1,2.3,22.9,0.0,0.0,0.0,47.0,53.0,9.0,13.0,5.0,33.0,37.0,9.0,13.0,12.5,0.665,0.156,0.454,0.055,0.000,0.335,0.614,0.794,0.556,0.583,0.000,0.356,0.449,0.885,0.055,12.0,Bigs,0.610


In [16]:
guards.head()

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group
5,LaMelo Ball,23,CHO,PG,15,15,33.1,9.9,23.1,0.428,4.5,12.5,0.356,5.4,10.5,0.513,0.525,4.7,5.3,0.886,1.0,6.7,4.2,28.9,73.0,26.0,1.0,0.0,0.0,35.0,11.0,8.0,33.0,253.0,7.0,15.0,17.6,0.457,0.173,0.182,0.081,0.020,0.543,0.513,0.617,0.492,0.429,0.143,0.356,0.235,0.567,0.009,2.0,Guards
6,De'Aaron Fox,27,SAC,PG,16,16,37.8,10.5,20.8,0.506,2.1,6.3,0.340,8.4,14.5,0.578,0.557,5.7,6.8,0.835,1.0,5.8,3.6,28.8,91.0,9.0,0.0,0.0,0.0,21.0,29.0,4.0,55.0,228.0,8.0,17.0,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015,5.0,Guards
8,Luka Dončić,25,DAL,PG,14,14,36.6,9.9,22.6,0.435,3.2,9.9,0.324,6.6,12.7,0.522,0.506,5.1,6.6,0.783,0.6,7.6,3.3,28.1,79.0,21.0,0.0,0.0,0.0,32.0,12.0,1.0,45.0,251.0,10.0,7.0,16.5,0.562,0.104,0.230,0.167,0.060,0.438,0.522,0.788,0.534,0.415,0.316,0.324,0.323,0.489,0.000,0.0,Guards
13,Jalen Brunson,28,NYK,PG,15,15,34.3,8.7,18.3,0.476,2.5,6.3,0.404,6.2,12.1,0.514,0.545,5.2,6.3,0.830,0.5,7.3,2.3,25.2,100.0,0.0,0.0,0.0,0.0,23.0,9.0,3.0,38.0,257.0,13.0,14.0,14.3,0.658,0.175,0.225,0.167,0.091,0.342,0.514,0.667,0.468,0.413,0.520,0.404,0.237,0.447,0.000,0.0,Guards
14,Tyrese Maxey,24,PHI,PG,9,9,36.0,9.0,21.7,0.415,2.8,9.9,0.281,6.2,11.8,0.528,0.479,4.4,5.3,0.833,0.1,3.8,1.7,25.2,64.0,36.0,0.0,0.0,0.0,6.0,3.0,2.0,18.0,88.0,7.0,10.0,16.0,0.544,0.179,0.221,0.077,0.067,0.456,0.528,0.571,0.512,0.667,0.308,0.281,0.196,0.320,0.026,5.0,Guards


In [17]:
wings['%FGA_Astd'] = wings['%FG_Astd_2P'] + wings['%FG_Astd_3P']
wings.fillna(0, inplace=True)
wings.head()

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_Astd
0,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,6.4,3.2,32.4,0.0,0.0,0.0,63.0,37.0,15.0,16.0,11.0,84.0,254.0,27.0,19.0,6.5,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.379,0.000,0.190,59.0,Wings,0.379
3,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,6.1,2.9,29.1,0.0,2.0,48.0,51.0,0.0,23.0,15.0,6.0,45.0,256.0,8.0,14.0,17.9,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.333,0.492,0.050,15.0,Wings,0.825
7,Shai Gilgeous-Alexander,26,OKC,SG,16,16,34.1,9.9,19.9,0.500,1.8,5.4,0.337,8.1,14.5,0.560,0.546,7.0,7.8,0.896,0.9,6.3,2.6,28.7,0.0,11.0,59.0,28.0,2.0,25.0,10.0,0.0,52.0,254.0,10.0,12.0,12.1,0.730,0.258,0.252,0.167,0.053,0.270,0.560,0.768,0.400,0.528,0.412,0.337,0.208,0.207,0.022,7.0,Wings,0.415
9,Anthony Edwards,23,MIN,SG,15,15,37.1,9.9,20.9,0.475,4.8,11.1,0.431,5.1,9.8,0.524,0.589,3.3,4.3,0.766,0.5,3.7,3.1,27.9,2.0,69.0,28.0,0.0,0.0,20.0,16.0,3.0,35.0,133.0,11.0,10.0,17.6,0.468,0.159,0.156,0.061,0.092,0.532,0.524,0.640,0.531,0.316,0.448,0.431,0.312,0.500,0.029,8.0,Wings,0.812
12,Jaylen Brown,28,BOS,SF,12,12,36.9,8.7,20.3,0.426,2.1,7.3,0.284,6.6,13.0,0.506,0.477,5.9,7.7,0.772,1.6,4.4,2.5,25.3,0.0,16.0,83.0,1.0,0.0,7.0,14.0,9.0,40.0,134.0,4.0,13.0,14.4,0.639,0.164,0.262,0.131,0.082,0.361,0.506,0.675,0.453,0.438,0.450,0.284,0.316,0.680,0.037,8.0,Wings,0.996


In [18]:
print(wings.isnull().sum())

Player            0
Age               0
Team              0
Pos               0
G                 0
GS                0
MP                0
FG                0
FGA               0
FG%               0
3P                0
3PA               0
3P%               0
2P                0
2PA               0
2P%               0
eFG%              0
FT                0
FTA               0
FT%               0
ORB               0
AST               0
TOV               0
PTS               0
PG%               0
SG%               0
SF%               0
PF%               0
C%                0
BadPassTO         0
LostBallTO        0
OffFoulCmtd       0
ShootFoulDrawn    0
PtsGenbyAsts      0
And1s             0
FGA_Blkd          0
AvgFGDist         0
%FGA_2P           0
%FGA_0-3          0
%FGA_3-10         0
%FGA_10-16        0
%FGA_16-3P        0
%FGA_3P           0
FG%_2P            0
FG%_0-3           0
FG%_3-10          0
FG%_10-16         0
FG%_16-3P         0
FG%_3P            0
%FG_Astd_2P       0


## Process

In [19]:
## Feature Engineering
bigs_features = bigs[['ORB','%FGA_3P', 'AvgFGDist', '%FGA_0-10', 'PTS', '%FGA_Dunks']]
guards_features = guards[['PTS', 'AST', 'PtsGenbyAsts', 'FGA', 'FTA']]
wings_features = wings[['PTS', '%FG_Astd_3P', '%FG_Astd_2P', 'FGA', 'FTA']]

In [20]:
scaler = StandardScaler()

bigs_scaled = pd.DataFrame(scaler.fit_transform(bigs_features), columns=bigs_features.columns)
guards_scaled = pd.DataFrame(scaler.fit_transform(guards_features), columns=guards_features.columns)
wings_scaled = pd.DataFrame(scaler.fit_transform(wings_features), columns=wings_features.columns)

## Analyze and Model

In [21]:
kmeans_bigs = KMeans(n_clusters=3, random_state=42, n_init=15)
bigs['Cluster'] = kmeans_bigs.fit_predict(bigs_scaled)

kmeans_guards = KMeans(n_clusters=3, random_state=42, n_init=15)
guards['Cluster'] = kmeans_guards.fit_predict(guards_scaled)

kmeans_wings = KMeans(n_clusters=3, random_state=42, n_init=15)
wings['Cluster'] = kmeans_wings.fit_predict(wings_scaled)

In [22]:
guards

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,Cluster
5,LaMelo Ball,23,CHO,PG,15,15,33.1,9.9,23.1,0.428,4.5,12.5,0.356,5.4,10.5,0.513,0.525,4.7,5.3,0.886,1.0,6.7,4.2,28.9,73.0,26.0,1.0,0.0,0.0,35.0,11.0,8.0,33.0,253.0,7.0,15.0,17.6,0.457,0.173,0.182,0.081,0.020,0.543,0.513,0.617,0.492,0.429,0.143,0.356,0.235,0.567,0.009,2.0,Guards,0
6,De'Aaron Fox,27,SAC,PG,16,16,37.8,10.5,20.8,0.506,2.1,6.3,0.340,8.4,14.5,0.578,0.557,5.7,6.8,0.835,1.0,5.8,3.6,28.8,91.0,9.0,0.0,0.0,0.0,21.0,29.0,4.0,55.0,228.0,8.0,17.0,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015,5.0,Guards,0
8,Luka Dončić,25,DAL,PG,14,14,36.6,9.9,22.6,0.435,3.2,9.9,0.324,6.6,12.7,0.522,0.506,5.1,6.6,0.783,0.6,7.6,3.3,28.1,79.0,21.0,0.0,0.0,0.0,32.0,12.0,1.0,45.0,251.0,10.0,7.0,16.5,0.562,0.104,0.230,0.167,0.060,0.438,0.522,0.788,0.534,0.415,0.316,0.324,0.323,0.489,0.000,0.0,Guards,0
13,Jalen Brunson,28,NYK,PG,15,15,34.3,8.7,18.3,0.476,2.5,6.3,0.404,6.2,12.1,0.514,0.545,5.2,6.3,0.830,0.5,7.3,2.3,25.2,100.0,0.0,0.0,0.0,0.0,23.0,9.0,3.0,38.0,257.0,13.0,14.0,14.3,0.658,0.175,0.225,0.167,0.091,0.342,0.514,0.667,0.468,0.413,0.520,0.404,0.237,0.447,0.000,0.0,Guards,0
14,Tyrese Maxey,24,PHI,PG,9,9,36.0,9.0,21.7,0.415,2.8,9.9,0.281,6.2,11.8,0.528,0.479,4.4,5.3,0.833,0.1,3.8,1.7,25.2,64.0,36.0,0.0,0.0,0.0,6.0,3.0,2.0,18.0,88.0,7.0,10.0,16.0,0.544,0.179,0.221,0.077,0.067,0.456,0.528,0.571,0.512,0.667,0.308,0.281,0.196,0.320,0.026,5.0,Guards,0
15,Damian Lillard,34,MIL,PG,13,13,35.8,7.7,17.8,0.431,2.8,8.9,0.319,4.8,8.9,0.543,0.511,6.5,7.2,0.914,0.7,7.6,2.8,24.8,100.0,0.0,0.0,0.0,0.0,25.0,10.0,0.0,38.0,234.0,3.0,11.0,17.6,0.500,0.181,0.112,0.086,0.121,0.500,0.543,0.667,0.346,0.600,0.500,0.319,0.143,0.459,0.004,1.0,Guards,0
16,Donovan Mitchell,28,CLE,SG,16,16,31.0,8.7,18.7,0.465,3.6,8.9,0.401,5.1,9.8,0.522,0.560,3.4,4.1,0.831,0.6,3.9,1.9,24.3,41.0,59.0,0.0,0.0,0.0,10.0,14.0,3.0,25.0,156.0,6.0,8.0,17.6,0.525,0.154,0.127,0.094,0.151,0.475,0.522,0.674,0.605,0.321,0.422,0.401,0.220,0.544,0.007,1.0,Guards,2
17,Tyler Herro,25,MIA,SG,13,13,33.8,8.2,16.9,0.486,4.4,9.7,0.452,3.8,7.2,0.532,0.616,3.4,3.8,0.880,0.3,5.2,2.6,24.2,21.0,78.0,1.0,0.0,0.0,15.0,12.0,4.0,23.0,168.0,5.0,13.0,17.5,0.427,0.141,0.214,0.059,0.014,0.573,0.532,0.677,0.447,0.462,0.667,0.452,0.340,0.719,0.014,3.0,Guards,2
23,Cade Cunningham,23,DET,PG,17,17,36.4,8.8,20.0,0.438,2.2,6.2,0.358,6.5,13.8,0.474,0.494,3.7,4.6,0.808,1.1,8.9,4.6,23.5,92.0,8.0,0.0,0.0,0.0,49.0,22.0,3.0,37.0,365.0,9.0,24.0,12.8,0.688,0.212,0.291,0.129,0.056,0.312,0.474,0.528,0.455,0.455,0.421,0.358,0.243,0.605,0.021,3.0,Guards,0
28,Stephen Curry,36,GSW,PG,12,12,29.6,7.6,15.5,0.489,4.1,9.1,0.450,3.5,6.4,0.545,0.621,3.4,3.6,0.953,0.3,6.6,3.7,22.7,100.0,0.0,0.0,0.0,0.0,33.0,6.0,3.0,13.0,182.0,5.0,9.0,18.4,0.414,0.140,0.177,0.059,0.038,0.586,0.545,0.615,0.394,0.727,0.714,0.450,0.429,0.633,0.000,0.0,Guards,2


In [23]:
wings

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_Astd,Cluster
0,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,6.4,3.2,32.4,0.0,0.0,0.0,63.0,37.0,15.0,16.0,11.0,84.0,254.0,27.0,19.0,6.5,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.379,0.000,0.190,59.0,Wings,0.379,1
3,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,6.1,2.9,29.1,0.0,2.0,48.0,51.0,0.0,23.0,15.0,6.0,45.0,256.0,8.0,14.0,17.9,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.333,0.492,0.050,15.0,Wings,0.825,1
7,Shai Gilgeous-Alexander,26,OKC,SG,16,16,34.1,9.9,19.9,0.500,1.8,5.4,0.337,8.1,14.5,0.560,0.546,7.0,7.8,0.896,0.9,6.3,2.6,28.7,0.0,11.0,59.0,28.0,2.0,25.0,10.0,0.0,52.0,254.0,10.0,12.0,12.1,0.730,0.258,0.252,0.167,0.053,0.270,0.560,0.768,0.400,0.528,0.412,0.337,0.208,0.207,0.022,7.0,Wings,0.415,1
9,Anthony Edwards,23,MIN,SG,15,15,37.1,9.9,20.9,0.475,4.8,11.1,0.431,5.1,9.8,0.524,0.589,3.3,4.3,0.766,0.5,3.7,3.1,27.9,2.0,69.0,28.0,0.0,0.0,20.0,16.0,3.0,35.0,133.0,11.0,10.0,17.6,0.468,0.159,0.156,0.061,0.092,0.532,0.524,0.640,0.531,0.316,0.448,0.431,0.312,0.500,0.029,8.0,Wings,0.812,1
12,Jaylen Brown,28,BOS,SF,12,12,36.9,8.7,20.3,0.426,2.1,7.3,0.284,6.6,13.0,0.506,0.477,5.9,7.7,0.772,1.6,4.4,2.5,25.3,0.0,16.0,83.0,1.0,0.0,7.0,14.0,9.0,40.0,134.0,4.0,13.0,14.4,0.639,0.164,0.262,0.131,0.082,0.361,0.506,0.675,0.453,0.438,0.450,0.284,0.316,0.680,0.037,8.0,Wings,0.996,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,Cam Reddish,25,LAL,SF,9,6,21.4,1.8,2.8,0.640,0.8,1.2,0.636,1.0,1.6,0.643,0.780,0.7,1.1,0.600,0.7,0.6,0.3,5.0,0.0,26.0,55.0,20.0,0.0,0.0,2.0,1.0,4.0,13.0,0.0,2.0,12.1,0.560,0.440,0.080,0.040,0.000,0.440,0.643,0.818,0.000,0.000,0.000,0.636,0.444,1.000,0.080,1.0,Wings,1.444,0
289,Pat Connaughton,32,MIL,SG,13,0,17.6,1.7,4.8,0.355,0.9,3.1,0.300,0.8,1.7,0.455,0.452,0.6,0.8,0.800,0.5,1.8,0.8,4.9,2.0,37.0,60.0,2.0,0.0,9.0,1.0,0.0,3.0,56.0,0.0,5.0,17.7,0.355,0.210,0.129,0.016,0.000,0.645,0.455,0.615,0.250,0.000,0.000,0.300,1.000,0.917,0.016,1.0,Wings,1.917,0
290,Pelle Larsson,23,MIA,SG,8,0,13.1,1.8,3.3,0.538,0.6,1.5,0.417,1.1,1.8,0.643,0.635,0.8,1.3,0.600,0.3,1.4,0.5,4.9,14.0,39.0,45.0,3.0,0.0,2.0,1.0,0.0,5.0,29.0,2.0,1.0,13.3,0.538,0.269,0.269,0.000,0.000,0.462,0.643,0.714,0.571,0.000,0.000,0.417,0.778,1.000,0.038,1.0,Wings,1.778,0
292,Dalen Terry,22,CHI,SG,17,0,13.8,1.6,3.2,0.491,0.5,1.1,0.444,1.1,2.2,0.514,0.564,1.2,1.6,0.778,0.8,1.1,1.0,4.9,2.0,21.0,72.0,5.0,0.0,9.0,3.0,0.0,12.0,48.0,1.0,4.0,10.0,0.673,0.527,0.109,0.000,0.036,0.327,0.514,0.552,0.333,0.000,0.500,0.444,0.579,1.000,0.055,3.0,Wings,1.579,0


In [24]:
bigs

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_0-10,Cluster
1,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,2.8,2.1,31.3,0.0,0.0,0.0,6.0,94.0,14.0,10.0,2.0,60.0,95.0,15.0,14.0,8.2,0.888,0.378,0.326,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0,Bigs,0.704,2
2,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,11.5,4.0,30.0,0.0,0.0,0.0,0.0,100.0,22.0,17.0,2.0,28.0,303.0,3.0,14.0,9.2,0.808,0.280,0.453,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0,Bigs,0.733,2
10,Kevin Durant,36,PHO,PF,9,9,38.8,9.9,17.9,0.553,2.7,6.2,0.429,7.2,11.7,0.619,0.627,5.1,6.1,0.836,0.2,3.4,3.3,27.6,0.0,0.0,0.0,84.0,16.0,17.0,6.0,1.0,26.0,76.0,9.0,3.0,15.0,0.652,0.155,0.130,0.292,0.075,0.348,0.619,0.920,0.667,0.468,0.500,0.429,0.492,0.750,0.037,6.0,Bigs,0.285,0
11,Karl-Anthony Towns,29,NYK,C,14,14,33.0,9.6,17.5,0.551,2.6,5.3,0.500,7.0,12.2,0.573,0.627,4.9,5.7,0.850,2.8,2.9,1.9,26.8,0.0,0.0,0.0,0.0,100.0,8.0,7.0,8.0,43.0,93.0,14.0,18.0,11.2,0.698,0.343,0.278,0.061,0.016,0.302,0.573,0.631,0.559,0.467,0.000,0.500,0.541,0.973,0.073,17.0,Bigs,0.621,2
27,Jaren Jackson Jr.,25,MEM,C,14,14,28.9,8.2,15.6,0.528,1.9,5.2,0.356,6.4,10.4,0.614,0.587,4.6,5.8,0.802,1.3,1.1,2.3,22.9,0.0,0.0,0.0,47.0,53.0,9.0,13.0,5.0,33.0,37.0,9.0,13.0,12.5,0.665,0.156,0.454,0.055,0.000,0.335,0.614,0.794,0.556,0.583,0.000,0.356,0.449,0.885,0.055,12.0,Bigs,0.610,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Donovan Clingan,20,POR,C,16,5,16.5,2.5,4.5,0.556,0.2,0.6,0.333,2.3,3.9,0.587,0.576,1.0,1.4,0.727,2.8,0.3,1.0,6.2,0.0,0.0,0.0,0.0,100.0,6.0,5.0,4.0,13.0,9.0,4.0,8.0,4.7,0.875,0.806,0.056,0.014,0.000,0.125,0.587,0.638,0.000,0.000,0.000,0.333,0.541,1.000,0.306,16.0,Bigs,0.862,1
277,Kevon Looney,28,GSW,C,13,0,15.5,2.2,4.4,0.491,0.0,0.0,0.000,2.2,4.4,0.491,0.491,1.1,2.0,0.538,3.9,1.7,0.6,5.4,0.0,0.0,0.0,0.0,100.0,2.0,3.0,3.0,9.0,55.0,2.0,2.0,4.0,1.000,0.596,0.263,0.053,0.088,0.000,0.491,0.618,0.400,0.000,0.200,0.000,0.429,0.000,0.088,5.0,Bigs,0.859,1
291,Mason Plumlee,34,PHO,C,15,3,17.6,1.9,3.3,0.580,0.0,0.0,0.000,1.9,3.3,0.580,0.580,1.1,2.0,0.533,1.6,1.8,0.9,4.9,0.0,0.0,0.0,4.0,96.0,1.0,4.0,6.0,10.0,65.0,2.0,11.0,3.0,1.000,0.580,0.380,0.040,0.000,0.000,0.580,0.828,0.263,0.000,0.000,0.000,0.724,0.000,0.260,12.0,Bigs,0.960,1
296,Drew Eubanks,27,UTA,C,12,0,14.9,1.9,3.0,0.639,0.1,0.3,0.333,1.8,2.8,0.667,0.653,0.8,1.3,0.600,1.4,1.2,0.8,4.7,0.0,0.0,0.0,10.0,90.0,3.0,4.0,1.0,7.0,35.0,0.0,1.0,7.1,0.917,0.333,0.472,0.111,0.000,0.083,0.667,0.750,0.588,0.750,0.000,0.333,0.818,0.000,0.194,6.0,Bigs,0.805,1


In [25]:
guardscorer = guards[guards['Cluster'] == 1]
guardbal = guards[guards['Cluster'] == 2]
guardpass = guards[guards['Cluster'] == 0]

wingset = wings[wings['Cluster'] == 0]
wingiso = wings[wings['Cluster'] == 1]
wingbal = wings[wings['Cluster'] == 2]

bigb2b = bigs[bigs['Cluster'] == 1]
bigbal = bigs[bigs['Cluster'] == 2]
bigstretch = bigs[bigs['Cluster'] == 0]

In [26]:
Sguards_score = guardscorer.sort_values(by='PTS', ascending=False)
Sguards_bal = guardbal.sort_values(by='PTS', ascending=False)
Sguards_pass = guardpass.sort_values(by='AST', ascending=False)
Swings_set = wingset.sort_values(by='PTS', ascending=False)
Swings_iso = wingiso.sort_values(by='PTS', ascending=False)
Swings_bal = wingbal.sort_values(by='PTS', ascending=False)
Sbigs_b2b = bigb2b.sort_values(by='PTS', ascending=False)
Sbigs_bal = bigbal.sort_values(by='PTS', ascending=False)
Sbigs_stretch = bigstretch.sort_values(by='PTS', ascending=False)

## Share

In [27]:
print("\nScoring Guards:")
Sguards_score[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3P%', '2P%', 'eFG%', 'FT%', 'PTS',
       'AST', 'TOV', 'ShootFoulDrawn',
       'And1s', 'FGA_Blkd', 'AvgFGDist', '%FGA_2P', '%FGA_0-3',
       '%FGA_3-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', '%FG_Astd_2P',
       '%FG_Astd_3P', 'Num_Dunks']]


Scoring Guards:


,Player,Age,Team,Pos,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,PTS,AST,TOV,ShootFoulDrawn,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,%FG_Astd_2P,%FG_Astd_3P,Num_Dunks
74,Jared McCain,20,PHI,SG,15,5,23.7,0.484,0.427,0.538,0.588,0.971,16.5,2.7,1.3,15.0,3.0,9.0,15.8,0.511,0.209,0.176,0.099,0.027,0.489,0.280,0.816,2.0
96,Payton Pritchard,27,BOS,PG,16,0,27.7,0.470,0.421,0.657,0.637,0.862,14.9,2.9,1.1,8.0,2.0,8.0,22.9,0.208,0.083,0.048,0.048,0.030,0.792,0.174,0.839,0.0
104,Tre Mann,23,CHO,PG,13,0,24.5,0.435,0.400,0.455,0.509,0.905,14.1,3.0,1.9,9.0,3.0,13.0,13.6,0.627,0.273,0.205,0.099,0.050,0.373,0.174,0.542,1.0
112,Jrue Holiday,34,BOS,PG,15,15,30.5,0.489,0.387,0.609,0.594,0.903,12.9,3.8,1.6,11.0,2.0,6.0,15.8,0.460,0.194,0.223,0.022,0.022,0.540,0.513,0.828,0.0
113,Terry Rozier,30,MIA,PG,12,12,31.8,0.380,0.333,0.429,0.465,0.885,12.9,3.7,1.8,13.0,5.0,8.0,15.9,0.493,0.197,0.183,0.049,0.063,0.507,0.600,0.750,0.0
117,Malik Monk,26,SAC,SG,10,0,25.0,0.442,0.279,0.557,0.500,0.846,12.6,2.8,0.8,12.0,2.0,7.0,15.2,0.587,0.163,0.260,0.115,0.048,0.413,0.441,0.750,4.0
155,Jose Alvarado,26,NOP,PG,11,7,25.6,0.396,0.400,0.390,0.515,0.824,10.7,4.5,1.3,7.0,0.0,4.0,18.5,0.406,0.079,0.178,0.109,0.040,0.594,0.375,0.625,0.0
156,Miles McBride,24,NYK,PG,11,0,26.4,0.440,0.379,0.524,0.550,0.800,10.7,2.5,1.1,5.0,1.0,5.0,18.0,0.420,0.140,0.150,0.080,0.050,0.580,0.364,0.727,2.0
159,Stephon Castle,20,SAS,PG,16,9,25.8,0.387,0.281,0.452,0.440,0.723,10.4,3.6,1.7,23.0,5.0,11.0,14.4,0.620,0.293,0.167,0.067,0.093,0.380,0.595,0.813,10.0
161,Grayson Allen,29,PHO,SG,12,0,21.8,0.376,0.350,0.476,0.515,0.769,10.3,1.0,1.3,11.0,1.0,3.0,22.0,0.208,0.040,0.119,0.050,0.000,0.792,0.900,0.964,0.0


In [28]:
print("\nBalanced Guards:")
Sguards_bal[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3P%', '2P%', 'eFG%', 'FT%',
       'ORB', 'PTS', 'AST', 'TOV',
       'ShootFoulDrawn',
       'PtsGenbyAsts', 'And1s', 'FGA_Blkd', 'AvgFGDist', '%FGA_2P', '%FGA_0-3',
       '%FGA_3-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P',
       '%FG_Astd_3P', 'Num_Dunks']]


Balanced Guards:


,Player,Age,Team,Pos,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,PTS,AST,TOV,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,Num_Dunks
16,Donovan Mitchell,28,CLE,SG,16,16,31.0,0.465,0.401,0.522,0.560,0.831,0.6,24.3,3.9,1.9,25.0,156.0,6.0,8.0,17.6,0.525,0.154,0.127,0.094,0.151,0.475,0.522,0.674,0.605,0.321,0.422,0.401,0.220,0.544,1.0
17,Tyler Herro,25,MIA,SG,13,13,33.8,0.486,0.452,0.532,0.616,0.880,0.3,24.2,5.2,2.6,23.0,168.0,5.0,13.0,17.5,0.427,0.141,0.214,0.059,0.014,0.573,0.532,0.677,0.447,0.462,0.667,0.452,0.340,0.719,3.0
28,Stephen Curry,36,GSW,PG,12,12,29.6,0.489,0.450,0.545,0.621,0.953,0.3,22.7,6.6,3.7,13.0,182.0,5.0,9.0,18.4,0.414,0.140,0.177,0.059,0.038,0.586,0.545,0.615,0.394,0.727,0.714,0.450,0.429,0.633,0.0
36,Jordan Poole,25,WAS,SG,14,14,30.1,0.459,0.443,0.474,0.566,0.889,0.3,21.2,4.9,3.2,18.0,157.0,2.0,17.0,15.6,0.518,0.245,0.182,0.064,0.027,0.482,0.474,0.611,0.400,0.143,0.500,0.443,0.315,0.574,2.0
38,Darius Garland,25,CLE,PG,16,16,30.3,0.492,0.430,0.540,0.586,0.935,0.5,20.6,7.0,2.6,20.0,261.0,3.0,9.0,15.8,0.561,0.197,0.148,0.127,0.090,0.439,0.540,0.646,0.500,0.613,0.273,0.430,0.270,0.630,0.0
44,Coby White,24,CHI,SG,17,17,33.1,0.445,0.381,0.554,0.565,0.883,0.2,19.5,4.8,2.6,25.0,202.0,5.0,8.0,19.5,0.372,0.162,0.093,0.061,0.057,0.628,0.554,0.675,0.478,0.733,0.143,0.381,0.333,0.831,0.0
47,Jalen Green,22,HOU,SG,17,17,32.2,0.390,0.319,0.457,0.467,0.857,0.6,19.2,2.8,2.6,28.0,107.0,5.0,24.0,16.3,0.517,0.199,0.171,0.045,0.103,0.483,0.457,0.707,0.340,0.154,0.300,0.319,0.304,0.511,13.0
58,Jamal Murray,27,DEN,PG,11,11,36.5,0.397,0.338,0.440,0.468,0.850,0.5,17.9,6.0,2.2,14.0,163.0,3.0,11.0,17.4,0.575,0.109,0.161,0.132,0.172,0.425,0.440,0.684,0.429,0.391,0.333,0.338,0.409,0.760,2.0
68,Dennis Schröder,31,BRK,PG,16,16,34.3,0.448,0.408,0.486,0.547,0.843,0.1,17.2,6.5,2.4,15.0,261.0,2.0,14.0,16.1,0.514,0.217,0.160,0.085,0.052,0.486,0.486,0.609,0.324,0.500,0.455,0.408,0.208,0.690,0.0
85,Anfernee Simons,25,POR,SG,13,13,29.1,0.378,0.300,0.453,0.451,0.923,0.2,15.6,3.9,2.5,17.0,126.0,4.0,15.0,16.6,0.514,0.184,0.184,0.114,0.032,0.486,0.453,0.647,0.382,0.190,0.667,0.300,0.233,0.333,3.0


In [29]:
print("\nPlaymaking Guards:")
Sguards_pass[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3P%', '2P%', 'eFG%', 'FT%', 'PTS',
       'ORB', 'AST', 'TOV',
       'BadPassTO', 'LostBallTO',
       'PtsGenbyAsts', 'AvgFGDist', '%FGA_2P', '%FGA_0-3',
       '%FGA_3-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P',
       '%FG_Astd_3P', '%FGA_Dunks']]


Playmaking Guards:


,Player,Age,Team,Pos,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,PTS,ORB,AST,TOV,BadPassTO,LostBallTO,PtsGenbyAsts,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks
33,Trae Young,26,ATL,PG,16,16,36.1,0.384,0.328,0.439,0.466,0.898,22.1,0.8,11.7,4.6,59.0,12.0,423.0,17.2,0.504,0.127,0.217,0.120,0.040,0.496,0.439,0.600,0.350,0.394,0.545,0.328,0.344,0.400,0.000
39,Ja Morant,25,MEM,PG,8,8,27.9,0.448,0.258,0.511,0.480,0.833,20.6,0.5,9.1,4.3,23.0,6.0,172.0,11.5,0.752,0.264,0.328,0.104,0.056,0.248,0.511,0.667,0.463,0.308,0.429,0.258,0.208,0.750,0.016
23,Cade Cunningham,23,DET,PG,17,17,36.4,0.438,0.358,0.474,0.494,0.808,23.5,1.1,8.9,4.6,49.0,22.0,365.0,12.8,0.688,0.212,0.291,0.129,0.056,0.312,0.474,0.528,0.455,0.455,0.421,0.358,0.243,0.605,0.021
40,James Harden,35,LAC,PG,17,17,34.9,0.365,0.311,0.419,0.443,0.877,20.4,0.6,8.8,4.4,37.0,27.0,344.0,16.8,0.502,0.151,0.196,0.144,0.011,0.498,0.419,0.561,0.321,0.385,0.667,0.311,0.175,0.524,0.007
8,Luka Dončić,25,DAL,PG,14,14,36.6,0.435,0.324,0.522,0.506,0.783,28.1,0.6,7.6,3.3,32.0,12.0,251.0,16.5,0.562,0.104,0.230,0.167,0.060,0.438,0.522,0.788,0.534,0.415,0.316,0.324,0.323,0.489,0.000
15,Damian Lillard,34,MIL,PG,13,13,35.8,0.431,0.319,0.543,0.511,0.914,24.8,0.7,7.6,2.8,25.0,10.0,234.0,17.6,0.500,0.181,0.112,0.086,0.121,0.500,0.543,0.667,0.346,0.600,0.500,0.319,0.143,0.459,0.004
13,Jalen Brunson,28,NYK,PG,15,15,34.3,0.476,0.404,0.514,0.545,0.830,25.2,0.5,7.3,2.3,23.0,9.0,257.0,14.3,0.658,0.175,0.225,0.167,0.091,0.342,0.514,0.667,0.468,0.413,0.520,0.404,0.237,0.447,0.000
5,LaMelo Ball,23,CHO,PG,15,15,33.1,0.428,0.356,0.513,0.525,0.886,28.9,1.0,6.7,4.2,35.0,11.0,253.0,17.6,0.457,0.173,0.182,0.081,0.020,0.543,0.513,0.617,0.492,0.429,0.143,0.356,0.235,0.567,0.009
6,De'Aaron Fox,27,SAC,PG,16,16,37.8,0.506,0.340,0.578,0.557,0.835,28.8,1.0,5.8,3.6,21.0,29.0,228.0,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015
14,Tyrese Maxey,24,PHI,PG,9,9,36.0,0.415,0.281,0.528,0.479,0.833,25.2,0.1,3.8,1.7,6.0,3.0,88.0,16.0,0.544,0.179,0.221,0.077,0.067,0.456,0.528,0.571,0.512,0.667,0.308,0.281,0.196,0.320,0.026


In [30]:
print("\nCatch and Shoot Wings:")
Swings_set[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3P%', '2P%', 'eFG%', 'FT%',
       'PTS', 'ShootFoulDrawn',
       'And1s', 'FGA_Blkd', 'AvgFGDist', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_2P', '%FGA_0-3',
       '%FGA_3-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P',
       '%FGA_Dunks']]


Catch and Shoot Wings:


,Player,Age,Team,Pos,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,PTS,ShootFoulDrawn,And1s,FGA_Blkd,AvgFGDist,%FG_Astd_2P,%FG_Astd_3P,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FGA_Dunks
109,Klay Thompson,34,DAL,SF,16,16,28.4,0.381,0.359,0.424,0.500,1.000,13.1,5.0,1.0,6.0,20.5,0.750,0.979,0.335,0.030,0.091,0.132,0.081,0.665,0.424,0.667,0.278,0.462,0.438,0.359,0.000
111,Santi Aldama,24,MEM,PF,16,11,28.6,0.477,0.289,0.620,0.540,0.889,12.9,9.0,2.0,5.0,13.2,0.742,0.955,0.568,0.273,0.250,0.034,0.011,0.432,0.620,0.792,0.477,0.500,0.000,0.289,0.131
124,Ochai Agbaji,24,TOR,SG,16,14,32.1,0.520,0.446,0.576,0.617,0.700,12.4,11.0,4.0,11.0,12.2,0.735,0.931,0.567,0.373,0.160,0.033,0.000,0.433,0.576,0.643,0.500,0.200,0.000,0.446,0.053
130,Dalton Knecht,23,LAL,SF,15,5,22.7,0.520,0.461,0.617,0.663,0.923,11.7,6.0,2.0,6.0,17.9,0.690,0.943,0.382,0.171,0.130,0.041,0.041,0.618,0.617,0.905,0.313,0.400,0.600,0.461,0.098
138,Chris Boucher,32,TOR,PF,16,0,19.1,0.508,0.333,0.667,0.587,0.711,11.4,17.0,4.0,5.0,14.3,0.652,1.000,0.523,0.273,0.152,0.083,0.015,0.477,0.667,0.750,0.550,0.636,0.500,0.333,0.106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,Cam Reddish,25,LAL,SF,9,6,21.4,0.640,0.636,0.643,0.780,0.600,5.0,4.0,0.0,2.0,12.1,0.444,1.000,0.560,0.440,0.080,0.040,0.000,0.440,0.643,0.818,0.000,0.000,0.000,0.636,0.080
289,Pat Connaughton,32,MIL,SG,13,0,17.6,0.355,0.300,0.455,0.452,0.800,4.9,3.0,0.0,5.0,17.7,1.000,0.917,0.355,0.210,0.129,0.016,0.000,0.645,0.455,0.615,0.250,0.000,0.000,0.300,0.016
290,Pelle Larsson,23,MIA,SG,8,0,13.1,0.538,0.417,0.643,0.635,0.600,4.9,5.0,2.0,1.0,13.3,0.778,1.000,0.538,0.269,0.269,0.000,0.000,0.462,0.643,0.714,0.571,0.000,0.000,0.417,0.038
292,Dalen Terry,22,CHI,SG,17,0,13.8,0.491,0.444,0.514,0.564,0.778,4.9,12.0,1.0,4.0,10.0,0.579,1.000,0.673,0.527,0.109,0.000,0.036,0.327,0.514,0.552,0.333,0.000,0.500,0.444,0.055


In [31]:
print("\nBalanced Wings:")
Swings_bal[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3P%', '2P%', 'eFG%', 'FT%', 'PTS',
       'ORB', 'AST', 'TOV',
       'BadPassTO', 'LostBallTO', 'ShootFoulDrawn',
       'PtsGenbyAsts', 'And1s', 'FGA_Blkd', 'AvgFGDist', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_2P', '%FGA_0-3',
       '%FGA_3-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P',
       '%FG_Astd_3P', '%FGA_Dunks']]


Balanced Wings:


,Player,Age,Team,Pos,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,PTS,ORB,AST,TOV,BadPassTO,LostBallTO,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FG_Astd_2P,%FG_Astd_3P,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks
34,Jalen Williams,23,OKC,SG,16,16,32.6,0.517,0.390,0.571,0.576,0.818,21.8,1.1,5.2,2.1,16.0,15.0,25.0,204.0,8.0,12.0,12.7,0.463,0.813,0.697,0.277,0.188,0.162,0.070,0.303,0.571,0.747,0.373,0.545,0.474,0.390,0.463,0.813,0.100
37,Pascal Siakam,30,IND,PF,16,16,33.9,0.554,0.449,0.598,0.620,0.772,20.8,1.8,3.6,1.4,7.0,10.0,30.0,135.0,9.0,9.0,12.8,0.490,1.000,0.704,0.223,0.215,0.197,0.069,0.296,0.598,0.808,0.560,0.478,0.375,0.449,0.490,1.000,0.021
45,Jalen Johnson,23,ATL,SF,16,16,36.5,0.474,0.338,0.531,0.524,0.730,19.4,2.1,5.3,3.3,26.0,8.0,30.0,198.0,5.0,23.0,11.3,0.621,0.920,0.708,0.285,0.285,0.123,0.016,0.292,0.531,0.750,0.417,0.258,0.750,0.338,0.621,0.920,0.079
46,Cameron Johnson,28,BRK,PF,16,16,33.6,0.498,0.423,0.620,0.629,0.918,19.3,0.6,2.9,0.8,5.0,5.0,16.0,111.0,6.0,6.0,18.2,0.653,0.909,0.378,0.201,0.086,0.048,0.043,0.622,0.620,0.786,0.444,0.500,0.333,0.423,0.653,0.909,0.038
52,Brandon Miller,22,CHO,SF,11,11,32.1,0.413,0.368,0.479,0.522,0.947,18.6,0.6,3.8,2.4,11.0,11.0,10.0,105.0,3.0,10.0,18.7,0.600,0.872,0.408,0.123,0.162,0.095,0.028,0.592,0.479,0.727,0.345,0.471,0.200,0.368,0.600,0.872,0.050
53,Derrick White,30,BOS,SG,16,16,34.9,0.464,0.404,0.581,0.598,0.861,18.4,1.1,4.3,1.5,21.0,2.0,14.0,170.0,3.0,9.0,19.6,0.512,0.898,0.336,0.091,0.164,0.050,0.032,0.664,0.581,0.550,0.583,0.818,0.286,0.404,0.512,0.898,0.009
54,Michael Porter Jr.,26,DEN,SF,14,14,36.5,0.485,0.394,0.564,0.576,0.667,18.2,1.4,2.9,1.0,5.0,6.0,13.0,95.0,4.0,10.0,16.3,0.839,0.892,0.539,0.181,0.181,0.044,0.132,0.461,0.564,0.730,0.514,0.333,0.481,0.394,0.839,0.892,0.059
55,Gradey Dick,21,TOR,SG,16,16,33.1,0.422,0.343,0.492,0.502,0.908,18.1,0.6,2.3,1.9,13.0,14.0,27.0,85.0,3.0,9.0,16.3,0.833,0.973,0.530,0.191,0.139,0.135,0.065,0.470,0.492,0.500,0.500,0.484,0.467,0.343,0.833,0.973,0.013
57,Jaden Ivey,22,DET,SG,16,16,31.8,0.435,0.359,0.472,0.494,0.730,18.0,1.5,4.3,3.0,21.0,15.0,38.0,169.0,11.0,17.0,12.7,0.493,0.786,0.671,0.287,0.232,0.105,0.046,0.329,0.472,0.603,0.364,0.400,0.364,0.359,0.493,0.786,0.034
59,Bradley Beal,31,PHO,SG,9,9,34.8,0.475,0.383,0.565,0.570,0.808,17.8,0.7,3.7,1.3,8.0,4.0,9.0,80.0,1.0,6.0,17.1,0.543,0.739,0.508,0.148,0.139,0.098,0.123,0.492,0.565,0.889,0.412,0.500,0.400,0.383,0.543,0.739,0.025


In [32]:
print("\nIso Scoring Wings:")
Swings_iso[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'FG%',
       '3P%', '2P%', 'eFG%', 'FT%', 'PTS',
       'ORB', 'TOV',
       'OffFoulCmtd', 'ShootFoulDrawn',
       'And1s', 'FGA_Blkd', 'AvgFGDist', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_2P', '%FGA_0-3',
       '%FGA_3-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P',
       '%FGA_Dunks', 'Num_Dunks']]


Iso Scoring Wings:


,Player,Age,Team,Pos,G,GS,FG%,3P%,2P%,eFG%,FT%,PTS,ORB,TOV,OffFoulCmtd,ShootFoulDrawn,And1s,FGA_Blkd,AvgFGDist,%FG_Astd_2P,%FG_Astd_3P,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FGA_Dunks,Num_Dunks
0,Giannis Antetokounmpo,30,MIL,PF,15,15,0.614,0.182,0.629,0.617,0.596,32.4,2.1,3.2,11.0,84.0,27.0,19.0,6.5,0.379,0.000,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.190,59.0
3,Jayson Tatum,26,BOS,PF,16,16,0.455,0.371,0.555,0.556,0.806,29.1,0.4,2.9,6.0,45.0,8.0,14.0,17.9,0.333,0.492,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.050,15.0
7,Shai Gilgeous-Alexander,26,OKC,SG,16,16,0.500,0.337,0.560,0.546,0.896,28.7,0.9,2.6,0.0,52.0,10.0,12.0,12.1,0.208,0.207,0.730,0.258,0.252,0.167,0.053,0.270,0.560,0.768,0.400,0.528,0.412,0.337,0.022,7.0
9,Anthony Edwards,23,MIN,SG,15,15,0.475,0.431,0.524,0.589,0.766,27.9,0.5,3.1,3.0,35.0,11.0,10.0,17.6,0.312,0.500,0.468,0.159,0.156,0.061,0.092,0.532,0.524,0.640,0.531,0.316,0.448,0.431,0.029,8.0
12,Jaylen Brown,28,BOS,SF,12,12,0.426,0.284,0.506,0.477,0.772,25.3,1.6,2.5,9.0,40.0,4.0,13.0,14.4,0.316,0.680,0.639,0.164,0.262,0.131,0.082,0.361,0.506,0.675,0.453,0.438,0.450,0.284,0.037,8.0
18,Cam Thomas,23,BRK,SG,15,15,0.455,0.385,0.503,0.534,0.871,24.2,0.4,3.1,5.0,44.0,11.0,21.0,15.7,0.538,0.571,0.587,0.167,0.178,0.136,0.106,0.413,0.503,0.614,0.383,0.500,0.536,0.385,0.000,0.0
19,Devin Booker,28,PHO,SG,16,16,0.438,0.339,0.509,0.509,0.886,24.1,1.3,2.1,4.0,39.0,5.0,18.0,16.6,0.435,0.707,0.580,0.101,0.177,0.160,0.142,0.420,0.509,0.690,0.471,0.457,0.488,0.339,0.010,2.0
20,LeBron James,40,LAL,SF,15,15,0.518,0.430,0.562,0.590,0.728,24.0,0.8,3.7,3.0,37.0,13.0,14.0,13.4,0.411,0.649,0.663,0.278,0.227,0.078,0.078,0.337,0.562,0.704,0.500,0.500,0.300,0.430,0.051,13.0
21,Kyrie Irving,32,DAL,SG,15,15,0.531,0.512,0.541,0.616,0.868,23.9,1.1,1.9,2.0,24.0,7.0,9.0,14.3,0.283,0.581,0.669,0.165,0.272,0.150,0.083,0.331,0.541,0.690,0.522,0.500,0.381,0.512,0.000,0.0
22,RJ Barrett,24,TOR,SG,13,13,0.438,0.338,0.483,0.490,0.716,23.8,0.9,3.3,1.0,45.0,9.0,22.0,11.4,0.298,0.769,0.693,0.299,0.327,0.044,0.024,0.307,0.483,0.653,0.341,0.455,0.333,0.338,0.028,6.0


In [33]:
print("\nBack to Basket Bigs:")
Sbigs_b2b[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3PA','3P%', '2P%', 'eFG%', 'FTA', 'FT%', 'PTS',
       'ORB', 'TOV',
       'OffFoulCmtd', 'ShootFoulDrawn',
       'And1s', 'FGA_Blkd', 'AvgFGDist', '%FGA_2P',
       '%FGA_0-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P',
       '%FG_Astd_3P', '%FGA_Dunks', 'Num_Dunks']]


Back to Basket Bigs:


,Player,Age,Team,Pos,G,GS,MP,FG%,3PA,3P%,2P%,eFG%,FTA,FT%,PTS,ORB,TOV,OffFoulCmtd,ShootFoulDrawn,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks
94,Ivica Zubac,27,LAC,C,17,17,33.7,0.577,0.0,0.000,0.577,0.577,3.4,0.579,15.1,4.2,2.4,8.0,29.0,9.0,14.0,4.3,1.000,0.922,0.067,0.010,0.000,0.577,0.655,0.543,0.308,0.5,0.000,0.750,0.0,0.186,32.0
98,Jarrett Allen,26,CLE,C,17,17,29.5,0.685,0.0,0.000,0.685,0.685,3.7,0.762,14.4,2.3,1.1,2.0,27.0,7.0,9.0,3.2,1.000,0.937,0.042,0.021,0.000,0.685,0.779,0.590,0.167,0.0,0.000,0.837,0.0,0.343,44.0
116,Daniel Gafford,26,DAL,C,16,11,20.9,0.708,0.0,0.000,0.708,0.708,2.9,0.674,12.6,3.0,0.9,4.0,20.0,6.0,10.0,2.4,1.000,0.991,0.008,0.000,0.000,0.708,0.744,0.622,1.000,0.0,0.000,0.729,0.0,0.308,32.0
132,Clint Capela,30,ATL,C,17,17,22.7,0.621,0.0,0.000,0.621,0.621,2.6,0.545,11.6,3.4,0.9,6.0,22.0,4.0,17.0,2.7,1.000,0.971,0.029,0.000,0.000,0.621,0.677,0.512,0.500,0.0,0.000,0.782,0.0,0.300,37.0
143,Zach Edey,22,MEM,C,14,9,20.1,0.610,0.7,0.600,0.611,0.638,2.4,0.667,11.1,3.1,1.6,7.0,16.0,4.0,9.0,5.7,0.905,0.867,0.029,0.010,0.095,0.611,0.811,0.316,0.667,1.0,0.600,0.690,1.0,0.276,28.0
146,Rudy Gobert,32,MIN,C,15,15,33.1,0.639,0.0,0.000,0.639,0.639,3.4,0.804,11.0,3.2,1.9,9.0,24.0,6.0,16.0,2.5,1.000,0.958,0.041,0.000,0.000,0.639,0.773,0.407,0.000,0.0,0.000,0.774,0.0,0.474,39.0
157,Onyeka Okongwu,24,ATL,C,15,0,23.5,0.526,1.9,0.207,0.632,0.552,2.7,0.780,10.7,2.0,0.9,3.0,15.0,3.0,11.0,8.1,0.750,0.698,0.052,0.000,0.250,0.632,0.700,0.429,0.667,0.0,0.207,0.782,1.0,0.207,22.0
185,Walker Kessler,23,UTA,C,9,9,26.4,0.686,0.0,0.000,0.686,0.686,2.6,0.565,9.2,3.8,1.7,3.0,12.0,3.0,7.0,2.6,1.000,0.961,0.020,0.020,0.000,0.686,0.853,0.400,0.000,0.0,0.000,0.686,0.0,0.333,17.0
193,Nic Claxton,25,BRK,C,12,5,24.6,0.653,0.0,0.000,0.653,0.653,2.3,0.464,8.9,1.9,0.9,5.0,10.0,2.0,3.0,4.0,1.000,0.888,0.111,0.000,0.000,0.653,0.805,0.391,0.625,0.0,0.000,0.660,0.0,0.222,14.0
198,Jalen Duren,21,DET,C,15,15,24.8,0.747,0.0,0.000,0.747,0.747,2.5,0.526,8.8,3.7,1.7,7.0,16.0,2.0,5.0,1.9,1.000,1.000,0.000,0.000,0.000,0.747,0.810,0.529,0.000,0.0,0.000,0.625,0.0,0.493,35.0


In [34]:
print("\nBalanced Bigs:")
Sbigs_bal[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3PA','3P%', '2P%', 'eFG%', 'FTA', 'FT%', 'PTS',
       'ORB', 'TOV',
       'OffFoulCmtd', 'ShootFoulDrawn',
       'And1s', 'FGA_Blkd', 'AvgFGDist', '%FGA_2P',
       '%FGA_0-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P',
       '%FG_Astd_3P', '%FGA_Dunks', 'Num_Dunks']]


Balanced Bigs:


,Player,Age,Team,Pos,G,GS,MP,FG%,3PA,3P%,2P%,eFG%,FTA,FT%,PTS,ORB,TOV,OffFoulCmtd,ShootFoulDrawn,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks
1,Anthony Davis,31,LAL,C,14,14,35.2,0.573,2.1,0.433,0.591,0.597,10.9,0.783,31.3,2.6,2.1,2.0,60.0,15.0,14.0,8.2,0.888,0.704,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0
2,Nikola Jokić,29,DEN,C,11,11,38.3,0.565,3.7,0.585,0.561,0.621,7.0,0.831,30.0,4.6,4.0,2.0,28.0,3.0,14.0,9.2,0.808,0.733,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0
11,Karl-Anthony Towns,29,NYK,C,14,14,33.0,0.551,5.3,0.500,0.573,0.627,5.7,0.850,26.8,2.8,1.9,8.0,43.0,14.0,18.0,11.2,0.698,0.621,0.061,0.016,0.302,0.573,0.631,0.559,0.467,0.000,0.500,0.541,0.973,0.073,17.0
27,Jaren Jackson Jr.,25,MEM,C,14,14,28.9,0.528,5.2,0.356,0.614,0.587,5.8,0.802,22.9,1.3,2.3,5.0,33.0,9.0,13.0,12.5,0.665,0.610,0.055,0.000,0.335,0.614,0.794,0.556,0.583,0.000,0.356,0.449,0.885,0.055,12.0
35,Julius Randle,30,MIN,PF,15,15,34.3,0.502,4.9,0.365,0.572,0.564,6.5,0.814,21.7,1.9,3.0,9.0,44.0,8.0,17.0,12.2,0.662,0.511,0.119,0.032,0.338,0.572,0.782,0.474,0.346,0.571,0.365,0.446,0.815,0.064,14.0
41,Domantas Sabonis,28,SAC,C,14,14,36.7,0.630,2.5,0.400,0.688,0.671,4.5,0.857,20.4,2.8,3.6,9.0,27.0,5.0,5.0,9.1,0.798,0.670,0.064,0.064,0.202,0.688,0.746,0.644,0.545,0.636,0.400,0.600,1.000,0.110,18.0
42,Nikola Vučević,34,CHI,C,17,17,32.4,0.577,4.5,0.461,0.630,0.649,2.1,0.857,20.2,2.1,1.8,4.0,17.0,5.0,7.0,12.3,0.685,0.548,0.104,0.033,0.315,0.630,0.750,0.613,0.440,0.625,0.461,0.750,0.971,0.012,3.0
56,Alperen Sengun,22,HOU,C,17,17,29.2,0.484,1.6,0.296,0.507,0.500,4.4,0.747,18.1,3.5,2.1,6.0,34.0,11.0,27.0,7.7,0.893,0.714,0.147,0.032,0.107,0.507,0.563,0.473,0.541,0.125,0.296,0.447,1.000,0.063,12.0
71,Jakob Poeltl,29,TOR,C,16,16,32.7,0.608,0.0,0.000,0.608,0.608,3.5,0.643,17.0,4.3,2.6,9.0,24.0,6.0,15.0,4.3,1.000,0.892,0.098,0.010,0.000,0.608,0.689,0.500,0.632,0.000,0.000,0.695,0.000,0.036,5.0
76,Bam Adebayo,27,MIA,C,13,13,35.2,0.426,2.5,0.313,0.450,0.454,5.0,0.708,16.3,3.0,1.5,3.0,27.0,5.0,8.0,10.5,0.825,0.590,0.197,0.038,0.175,0.450,0.692,0.377,0.389,0.143,0.313,0.559,0.800,0.153,26.0


In [35]:
print("\nStretch Bigs:")
Sbigs_stretch[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       '3PA','3P%', '2P%', 'eFG%', 'FTA', 'FT%', 'PTS',
       'ORB', 'TOV',
       'OffFoulCmtd', 'ShootFoulDrawn',
       'And1s', 'FGA_Blkd', 'AvgFGDist', '%FGA_2P',
       '%FGA_0-10', '%FGA_10-16', '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3',
       'FG%_3-10', 'FG%_10-16', 'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P',
       '%FG_Astd_3P', '%FGA_Dunks', 'Num_Dunks']]


Stretch Bigs:


,Player,Age,Team,Pos,G,GS,MP,FG%,3PA,3P%,2P%,eFG%,FTA,FT%,PTS,ORB,TOV,OffFoulCmtd,ShootFoulDrawn,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks
10,Kevin Durant,36,PHO,PF,9,9,38.8,0.553,6.2,0.429,0.619,0.627,6.1,0.836,27.6,0.2,3.3,1.0,26.0,9.0,3.0,15.0,0.652,0.285,0.292,0.075,0.348,0.619,0.920,0.667,0.468,0.500,0.429,0.492,0.750,0.037,6.0
29,Victor Wembanyama,21,SAS,C,13,13,31.5,0.472,8.6,0.339,0.598,0.555,3.6,0.872,22.7,2.3,3.5,3.0,21.0,5.0,8.0,16.9,0.511,0.332,0.066,0.114,0.489,0.598,0.833,0.455,0.467,0.308,0.339,0.614,0.789,0.140,28.0
50,Lauri Markkanen,27,UTA,PF,12,12,31.4,0.450,6.3,0.427,0.474,0.556,5.5,0.864,18.8,1.8,1.4,3.0,24.0,7.0,8.0,16.1,0.503,0.390,0.046,0.066,0.497,0.474,0.516,0.571,0.286,0.200,0.427,0.583,1.000,0.086,10.0
70,OG Anunoby,27,NYK,PF,15,15,36.3,0.503,5.5,0.373,0.604,0.585,2.9,0.818,17.1,1.1,1.2,2.0,22.0,4.0,12.0,13.5,0.561,0.460,0.074,0.026,0.439,0.604,0.742,0.333,0.357,0.600,0.373,0.797,1.000,0.159,27.0
72,Myles Turner,28,IND,C,14,14,32.6,0.480,5.1,0.431,0.514,0.568,3.6,0.700,16.9,0.9,2.7,10.0,29.0,9.0,7.0,14.4,0.593,0.430,0.096,0.068,0.407,0.514,0.732,0.343,0.471,0.333,0.431,0.778,0.968,0.096,15.0
75,Chet Holmgren,22,OKC,C,10,10,26.5,0.505,3.7,0.378,0.568,0.568,4.9,0.776,16.4,1.6,2.3,3.0,24.0,6.0,6.0,11.8,0.667,0.531,0.108,0.027,0.333,0.568,0.750,0.211,0.417,1.000,0.378,0.667,1.000,0.153,17.0
95,Paul George,34,PHI,PF,8,8,28.9,0.383,6.8,0.278,0.491,0.453,3.4,0.815,14.9,0.9,3.1,3.0,13.0,2.0,2.0,18.7,0.495,0.215,0.140,0.140,0.505,0.491,0.750,0.400,0.533,0.400,0.278,0.231,0.533,0.019,1.0
99,Naz Reid,25,MIN,C,15,0,24.5,0.507,5.1,0.416,0.600,0.612,2.1,0.906,14.3,0.9,1.1,1.0,13.0,5.0,10.0,15.2,0.493,0.460,0.033,0.000,0.507,0.600,0.667,0.541,0.600,0.000,0.416,0.667,0.969,0.026,4.0
101,Tobias Harris,32,DET,PF,17,17,33.2,0.455,4.4,0.333,0.522,0.514,1.7,0.793,14.2,1.1,1.4,7.0,16.0,6.0,14.0,13.7,0.648,0.437,0.155,0.056,0.352,0.522,0.784,0.446,0.364,0.500,0.333,0.514,0.920,0.047,10.0
108,Bobby Portis,29,MIL,PF,15,1,23.9,0.472,3.1,0.319,0.527,0.514,1.4,0.762,13.3,1.9,1.4,2.0,11.0,5.0,9.0,13.0,0.736,0.427,0.163,0.146,0.264,0.527,0.721,0.364,0.517,0.423,0.319,0.478,0.933,0.006,1.0


## Final Insights
* In this project, I leveraged K-means clustering to categorize NBA players into distinct offensive archetypes based on their playing style and court positioning data from Basketball-Reference. The archetypes include scoring guards, playmaking guards, and balanced guards, as well as iso wings, catch-and-shoot wings, and balanced wings. For bigs, the clusters consisted of back-to-the-basket bigs, stretch bigs, and balanced bigs.
* By analyzing the percentage of time each player spends at a particular position, I found that 68 out of 262 eligible players (at least 8 games played in the 2024-25 season) fit the "catch and shoot wing" archetype. This insight reflects trends in modern basketball, where certain archetypes, like catch-and-shoot wings, are prevalent due to the evolving nature of the game.
* It's important to note that factors like injuries and the growing trend of positionless basketball played a role in some clustering outcomes, such as Shai Gilgeous-Alexander of the Oklahoma City Thunder being categorized as a wing despite traditionally being a point guard.
* This analysis showcases my ability to apply clustering techniques and interpret real-world data, while also critically analyzing the results to account for complexities like player injuries and team dynamics.